In [1]:
from random import choices, choice
from helpers import *

def MakeRandomKey(keysize = 16):
    return bytes( choices(range(0xFF), k = keysize) )

In [15]:
MakeRandomKey()

b'B\xb7!\x9an\x92\xd6\xa2\xad\x0b\xa4\x97\x8d\xfeW\x0b'

In [3]:
def EncryptionOracle(plaintext):
    key = MakeRandomKey(16)
    plaintext = MakeRandomKey(choice(range(5, 11)) ) + plaintext + MakeRandomKey( choice(range(5, 11)) )
    
    mode = choice(['MODE_ECB', 'MODE_CBC'])

    if mode == 'MODE_ECB':
        ciphertext = EncryptECB(plaintext, key)
    else:
        initialization = MakeRandomKey(16)
        ciphertext = EncryptCBC(plaintext, key, initialization)
    return mode, ciphertext

In [10]:
with open('paradise.txt', mode = 'rb') as file:
    plaintext = file.read()

print(plaintext[:1000].decode('ascii', 'ignore'))

Of Mans First Disobedience, and the Fruit
Of that Forbidden Tree, whose mortal tast
Brought Death into the World, and all our woe, With loss of EDEN, till one greater Man
Restore us, and regain the blissful Seat,
Sing Heavnly Muse, that on the secret top
Of OREB, or of SINAI, didst inspire
That Shepherd, who first taught the chosen Seed, In the Beginning how the Heavns and Earth
Rose out of CHAOS: Or if SION Hill
Delight thee more, and SILOAS Brook that flowd Fast by the Oracle of God; I thence
Invoke thy aid to my adventrous Song,
That with no middle flight intends to soar
Above th AONIAN Mount, while it pursues Things unattempted yet in Prose or Rhime.
And chiefly Thou O Spirit, that dost prefer
Before all Temples th upright heart and pure, Instruct me, for Thou knowst; Thou from the first Wast present, and with mighty wings outspread Dove-like satst brooding on the vast Abyss
And madst it pregnant: What in me is dark Illumine, what is low raise and support;
T


In [11]:
ciphertext = EncryptionOracle(plaintext)

In [12]:
def DetectECB(ciphertext):
    BLOCKSIZE = 16
    assert len(ciphertext) % BLOCKSIZE == 0
    best_offset = 0
    best_ratio = 1
    
    for offset in range(16):
        blocks = [ bytes(chunk) for chunk in Chunkerize(ciphertext[offset:], BLOCKSIZE, strict=False) ]
        unique_blocks = { *blocks }
        ratio = len(unique_blocks) / len(blocks)
        
        if ratio < best_ratio:
            best_offset = offset
            best_ratio = ratio
    
    return best_ratio

In [13]:
mode, ciphertext = EncryptionOracle(plaintext)
print( DetectECB(ciphertext) )

0.9960017769880053


In [14]:
# This test is succesful if no 'Failed to detect...' is printed

for n in range(100):
    mode, ciphertext = EncryptionOracle(plaintext)
    ratio = DetectECB(ciphertext)
    if mode == 'MODE_ECB' and ratio >= 0.999999:
        print(f'Failed to detect {mode} on iteration {n}.')
    elif mode == 'MODE_CBC' and ratio < 1:
        print(f'Failed to detect {mode} on iteration {n}.')